In [1]:
import numpy as np
import pandas as pd

#### COURSE DATA

In [2]:
df = pd.read_csv("course_data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902 entries, 0 to 1901
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      1902 non-null   int64  
 1   Course Title                    1902 non-null   object 
 2   Instructor                      1902 non-null   object 
 3   Rating                          1902 non-null   float64
 4   No of students who gave rating  1902 non-null   object 
 5   Hours to complete course        1902 non-null   object 
 6   No of Lectures                  1902 non-null   object 
 7   Level                           1902 non-null   object 
 8   Image link                      1902 non-null   object 
 9   Course URL                      1902 non-null   object 
 10  Description                     1902 non-null   object 
 11  Discounted Price                1902 non-null   object 
 12  Price                           19

In [4]:
df['No of students who gave rating']=pd.to_numeric(df['No of students who gave rating'].str.replace(",",""))

In [5]:
del df['Unnamed: 0']

In [6]:
l=['total hours','total hour']
df['Hours to complete course']=df['Hours to complete course'].str.replace('|'.join(l),'', regex=True).str.strip()

In [7]:
l1=['total min']
for i  in range(0,len(df['Hours to complete course'])):
    #print(i)
    if l1[0] in df['Hours to complete course'][i]:
        j = df['Hours to complete course'][i].split(" ")
        print(type(j[0]))
        df['Hours to complete course'][i]=int(j[0])/60

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


<ipython-input-7-5ec07f20f273>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Hours to complete course'][i]=int(j[0])/60


In [8]:
df=df.drop(df.index[[488]])

In [9]:
df=df.drop(df.index[[626]])

In [10]:
df=df.drop(df.index[[1638]])

In [11]:
df['Hours to complete course']=pd.to_numeric(df['Hours to complete course'])

In [12]:
l=['lectures']
df['No of Lectures']=df['No of Lectures'].str.replace('|'.join(l),'', regex=True).str.strip()
df['No of Lectures']=pd.to_numeric(df['No of Lectures'])

In [13]:
list1 = {"All Levels": 1,"Beginner":2,"Expert":3,"Intermediate":4,"Null":0}
df["Level"]=df["Level"].replace(list1)

In [14]:
df['Discounted Price']=df['Discounted Price'].str.replace('₹','')
df['Discounted Price']=df['Discounted Price'].str.replace(',','')
df['Price']=df['Price'].str.replace('₹','')
df['Price']=df['Price'].str.replace(',','')
df['Price']=pd.to_numeric(df['Price'])
df['Discounted Price']=pd.to_numeric(df['Discounted Price'])

In [15]:
df['Description']

0       From Beginner to iOS App Developer with Just O...
1       Learn Node.js by building real-world applicati...
2       Learn modern HTML5, CSS3 and web design by bui...
3       Become an In-demand SQL Master by creating com...
4       Spring 5: Learn Spring 5 Core, AOP, Spring MVC...
                              ...                        
1897    Go from Beginner to Advanced by using Node.js,...
1898    Master the important features and techniques o...
1899    Learn PHP Data Object Objects or PDO and make ...
1900                  Database Management Systems - MySQL
1901    This tutorial will provide a good foundation o...
Name: Description, Length: 1899, dtype: object

## Trial

In [19]:
import streamlit as st 
import streamlit.components.v1 as stc 


# Load EDA
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel




In [20]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [21]:
def vectorize_text_to_cosine_mat(data):
    count_vect = CountVectorizer()
    cv_mat = count_vect.fit_transform(data)
    # Get the cosine
    cosine_sim_mat = cosine_similarity(cv_mat)
    return cosine_sim_mat

In [23]:
def get_recommendation(title,cosine_sim_mat,df,num_of_rec=10):
    course_indices = pd.Series(df.index,index=df['Course Title']).drop_duplicates()
    #print(course_indices)
    # Index of course
    idx = course_indices[title]

# Look into the cosine matr for that index
    sim_scores =list(enumerate(cosine_sim_mat[idx]))
    sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
    selected_course_indices = [i[0] for i in sim_scores[1:]]
    selected_course_scores = [i[0] for i in sim_scores[1:]]
# Get the dataframe & title
    result_df = df.iloc[selected_course_indices]
    result_df['similarity_score'] = selected_course_scores
    final_recommended_courses = result_df[['Course Title','Description','Subcategory','Sub subcategory','Price','Discounted Price','similarity_score']]
    
    #print (final_recommended_courses.head(num_of_rec))
    return final_recommended_courses.head(num_of_rec)


In [49]:
cosine_sim_mat = vectorize_text_to_cosine_mat(df['Description'])

In [121]:
search_term = "Master Microservices with Spring Boot and Spring Cloud"

In [128]:
def initial_rec(search_item):
    result=pd.DataFrame()
    result=result.append(get_recommendation(search_item,cosine_sim_mat,df,5))
    result=result.sort_values(by=['similarity_score'],ascending=False)
    suggestion=pd.DataFrame()
    result=result[:5]
    suggestion1= result[['Course Title','Description','Subcategory','Sub subcategory','Price','Discounted Price']]
    suggestion1['Visited'] = np.random.randint(0, 2, suggestion1.shape[0])
    suggestion1['Enrolled'] = np.random.randint(0, 2, suggestion1.shape[0])
    #print(suggestion1)
    return suggestion1

In [93]:
result=pd.DataFrame()

In [96]:
def function_rec(choice):
    result=pd.DataFrame()
    for i in choice['Course Title']:
        result=result.append(get_recommendation(i,cosine_sim_mat,df,5))
    result.drop_duplicates(subset ="Course Title",keep = False, inplace = True)
    result=result.sort_values(by=['similarity_score'],ascending=False)
    suggestion=pd.DataFrame()
    result=result[:5]
    suggestion1= result[['Course Title','Description','Subcategory','Sub subcategory','Price','Discounted Price']]
    suggestion1['Visited'] = np.random.randint(0, 2, suggestion1.shape[0])
    suggestion1['Enrolled'] = np.random.randint(0, 2, suggestion1.shape[0])
    return suggestion1
    

In [147]:
def function_rec1(choice,user_course):
    result=pd.DataFrame()
    for i in choice['Course Title']:
        if i not in user_course:
            result=result.append(get_recommendation(i,cosine_sim_mat,df,5))
        else:
            continue
    result.drop_duplicates(subset ="Course Title",keep = False, inplace = True)
    print(result)
    result=result.sort_values(by=['similarity_score'],ascending=False)
    suggestion=pd.DataFrame()
    result=result[:5]
    suggestion1= result[['Course Title','Description','Subcategory','Sub subcategory','Price','Discounted Price']]
    suggestion1['Visited'] = np.random.randint(0, 2, suggestion1.shape[0])
    suggestion1['Enrolled'] = np.random.randint(0, 2, suggestion1.shape[0])
    return suggestion1

In [102]:
def selected(suggestion):
    choice=suggestion.loc[suggestion['Enrolled'] == 1]
    return choice

In [103]:
choice= selected(suggestion1)

In [116]:
def user_course_append(choice):
    user_course=[]
    for i in choice['Course Title']:
        user_course.append(i)
    return user_course

In [151]:
user_course=[]

In [152]:
first_rec=initial_rec(search_term)

<ipython-input-23-df69789ed789>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['similarity_score'] = selected_course_scores


In [153]:
choice=selected(first_rec)

In [154]:
user_course=user_course_append(choice)

In [155]:
function_rec(choice)

                                           Course Title  \
1001                          Angular 4 Java Developers   
994   Learn Spring 5, Boot 2, JPA, Thymeleaf, AOP, W...   
978   Building An E-Commerce Store Using Java Spring...   
653               Testing Spring Boot: Beginner to Guru   
419   Spring Boot Microservices with Spring Cloud Be...   

                                            Description  \
1001  Learn How to Build Spring Boot & Angular Appli...   
994   Master Modern Spring Boot, Spring Web MVC, AOP...   
978   Hands-on experience with Spring MVC, Spring Da...   
653   Become an Expert Testing Java and Spring Boot ...   
419   Learn to Master Spring Boot Microservices with...   

                Subcategory   Sub subcategory  Price  Discounted Price  \
1001        Web Development           Angular   8640               385   
994   Programming Languages  Spring Framework   8640               385   
978         Web Development  Spring Framework   6080               3

<ipython-input-23-df69789ed789>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['similarity_score'] = selected_course_scores
